In [1]:
import tensorflow as tf
import scipy as sc
import pandas as pd
import numpy as np
import time
from sklearn.metrics import *
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from keras.losses import binary_crossentropy
from keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import r2_score
from keras import backend as K
from sklearn.model_selection import train_test_split

In [2]:
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
#         print(input_shape)
#         print(self.units)
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff, 2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [3]:
data_dir = 'https://github.com/henrii1/Wind-Turbine-power-prediction-and-monitoring-using-XGboost-and-quantile-regression/blob/main/data/kelmarsh_02.xlsx?raw=true'
data = pd.read_excel(data_dir)
data.isna().sum()

def outlier_remover(dat, prop, min, max):
    d = dat
    q_low = d[prop].quantile(min)
    q_high = d[prop].quantile(max)
    return d[(d[prop]<q_high) & (d[prop]>q_low)]

d1 = {}
step = 50
i = 1
for x in range(20, 3100, step):
    d1[i] = data.iloc[((data['power']>=x)& (data['power']<x+step)).values]
    i = i + 1

d1[-2] = data.iloc[(data['power']>=2900).values]

for x in range(1, 62):
    if x <= 3:
        F = 0.95
    elif ((x > 3) and (x <= 10)):
        F = 0.9
    elif ((x > 10) and (x <= 20)):
        F = 0.92
    elif ((x > 20) and (x <= 30)):
        F = 0.96
    else:
        F = 0.985
    d1[x] = outlier_remover(d1[x], 'wind speed', 0.00001, F)


df = pd.DataFrame()
for infile in range (1, 62):
    data = d1[infile]
    df = df.append(data, ignore_index = True)

da = df.drop(columns=['date'])
da.dropna()
scaler = MinMaxScaler(feature_range =(0, 1))
data_ = scaler.fit_transform(da)
data_x = data_[:, :-1]
data_y = data_[:, -1]

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.001, random_state = 1)


In [4]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    print(f'xgboost_RMSE:{rmse}')
    print(f'xgboost_MAE:{mae}')
    print(f'xgboost_R_SCORE:{r2}')

In [5]:
best_xgb_model = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)


best_xgb_model.fit(x_train,y_train)
pred1 = best_xgb_model.predict(x_test)

xgboost_model = eval_metrics(y_test, pred1)
print(f' xgboost cummulative accuracy is: {xgboost_model}')

[18:40:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgboost_RMSE:0.024752246550036723
xgboost_MAE:0.019512270052812506
xgboost_R_SCORE:0.9953668417712919
 xgboost cummulative accuracy is: None


In [6]:
print(f'the ground truth result is: {y_test}')

the ground truth result is: [0.78256331 0.01013892 0.93909211 0.93171893 0.97838549 0.9377304
 0.05729306 0.96406324 0.30674558 0.88423513 0.98838939 0.52518533
 0.15899424 0.10595266 0.08948024 0.49998389 0.36971785 0.22523765
 0.604794   0.04048216 0.55971815 0.00409338 0.06031447 0.832428
 0.02997866 0.26498554 0.07152355 0.9884246  0.0168044  0.64804015
 0.2965188  0.14913114 0.9497351  0.05940566 0.98500799 0.22879337
 0.09548048 0.19285071 0.14934963 0.33025048 0.22191112 0.15581709
 0.14532238 0.98067324]


In [7]:
print(f"xgboost predicted results is: {pred1}")

xgboost predicted results is: [ 8.29985440e-01  1.77671611e-02  9.56698060e-01  9.61145520e-01
  9.46979105e-01  9.48828697e-01  6.03212416e-02  9.85562801e-01
  3.29594433e-01  9.06087399e-01  9.80348229e-01  5.13096571e-01
  1.51456267e-01  1.30639166e-01  8.83782506e-02  5.34057379e-01
  4.02792037e-01  2.07528979e-01  5.35880804e-01  7.01160431e-02
  6.09420657e-01 -4.75645065e-04  5.09648025e-02  7.93050408e-01
  2.49950886e-02  2.27256447e-01  9.86234546e-02  9.77386892e-01
  1.02862120e-02  6.48894072e-01  2.86471635e-01  1.75332725e-01
  9.29428935e-01  5.14818132e-02  9.83720958e-01  2.13099808e-01
  1.09760076e-01  1.88424677e-01  1.32155061e-01  3.44749600e-01
  2.63763487e-01  1.92096978e-01  1.46066099e-01  9.74765480e-01]


In [8]:
model1_1A = Sequential()
model1_1A.add(Dense(128, input_dim = (3)))
model1_1A.add(RBFLayer(64, 0.5))
model1_1A.add(Dense(1, activation='sigmoid', name= 'output'))
model1_1A.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               512       
                                                                 
 rbf_layer (RBFLayer)        (None, 64)                8192      
                                                                 
 output (Dense)              (None, 1)                 65        
                                                                 
Total params: 8,769
Trainable params: 8,769
Non-trainable params: 0
_________________________________________________________________


In [9]:
model1_1A.compile(optimizer='adam',
              loss=binary_crossentropy,
              metrics=[
                       tf.keras.metrics.RootMeanSquaredError(),
                       tf.keras.metrics.MeanAbsoluteError(),
                       ])

In [10]:
history1_1A = model1_1A.fit(x_train, y_train, epochs=50, batch_size=50, verbose=1)


rbf_pred1 = model1_1A.predict(x_test)
print(r2_score(y_test, rbf_pred1))

Epoch 1/50
868/868 [==============================] - 7s 7ms/step - loss: 0.4311 - root_mean_squared_error: 0.0767 - mean_absolute_error: 0.0420
Epoch 2/50
868/868 [==============================] - 4s 5ms/step - loss: 0.4180 - root_mean_squared_error: 0.0337 - mean_absolute_error: 0.0255
Epoch 3/50
868/868 [==============================] - 4s 5ms/step - loss: 0.4177 - root_mean_squared_error: 0.0327 - mean_absolute_error: 0.0246
Epoch 4/50
868/868 [==============================] - 4s 5ms/step - loss: 0.4176 - root_mean_squared_error: 0.0325 - mean_absolute_error: 0.0244
Epoch 5/50
868/868 [==============================] - 4s 5ms/step - loss: 0.4174 - root_mean_squared_error: 0.0320 - mean_absolute_error: 0.0239
Epoch 6/50
868/868 [==============================] - 4s 5ms/step - loss: 0.4173 - root_mean_squared_error: 0.0318 - mean_absolute_error: 0.0237
Epoch 7/50
868/868 [==============================] - 4s 5ms/step - loss: 0.4172 - root_mean_squared_error: 0.0313 - mean_absolute

In [11]:
print(f"the rbf predicted result is: {rbf_pred1}")

the rbf predicted result is: [[0.8244563 ]
 [0.017665  ]
 [0.9563736 ]
 [0.96626973]
 [0.95405173]
 [0.9421664 ]
 [0.06759223]
 [0.99061036]
 [0.32761842]
 [0.9064632 ]
 [0.9847482 ]
 [0.51535237]
 [0.14398035]
 [0.11593625]
 [0.09105694]
 [0.5413427 ]
 [0.3985762 ]
 [0.19811058]
 [0.51933676]
 [0.05579421]
 [0.62618315]
 [0.01702455]
 [0.05929714]
 [0.7894338 ]
 [0.03557494]
 [0.2250866 ]
 [0.08841112]
 [0.9856474 ]
 [0.01540679]
 [0.6609605 ]
 [0.27940318]
 [0.17999005]
 [0.9348607 ]
 [0.0466634 ]
 [0.98759866]
 [0.2155413 ]
 [0.10033807]
 [0.17859015]
 [0.13057622]
 [0.33680886]
 [0.253179  ]
 [0.18505105]
 [0.14706418]
 [0.9760124 ]]


In [16]:
model1_2A = Sequential()
model1_2A.add(Dense(3, activation='relu', input_dim=(3)))
model1_2A.add(Dense(32, activation='relu'))
model1_2A.add(Dense(64, activation='relu'))
model1_2A.add(Dense(32, activation='relu'))
model1_2A.add(Dense(1, name='output'))
model1_2A.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 3)                 12        
                                                                 
 dense_7 (Dense)             (None, 32)                128       
                                                                 
 dense_8 (Dense)             (None, 64)                2112      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 output (Dense)              (None, 1)                 33        
                                                                 
Total params: 4,365
Trainable params: 4,365
Non-trainable params: 0
_________________________________________________________________


In [17]:
model1_2A.compile(optimizer='adam',
              loss=binary_crossentropy,
              metrics=[
                       tf.keras.metrics.RootMeanSquaredError(),
                       tf.keras.metrics.MeanAbsoluteError(),
                       ])

history1_2A = model1_2A.fit(x_train, y_train, epochs=50, batch_size=50, verbose=1)

y_pred1_2A = model1_2A.predict(x_test)


Epoch 1/50
868/868 [==============================] - 2s 2ms/step - loss: 2.6788 - root_mean_squared_error: 0.3741 - mean_absolute_error: 0.2696
Epoch 2/50
868/868 [==============================] - 2s 2ms/step - loss: 0.4784 - root_mean_squared_error: 0.1619 - mean_absolute_error: 0.1123
Epoch 3/50
868/868 [==============================] - 2s 2ms/step - loss: 0.4530 - root_mean_squared_error: 0.1228 - mean_absolute_error: 0.0768
Epoch 4/50
868/868 [==============================] - 2s 2ms/step - loss: 0.4724 - root_mean_squared_error: 0.1506 - mean_absolute_error: 0.0867
Epoch 5/50
868/868 [==============================] - 2s 2ms/step - loss: 0.4350 - root_mean_squared_error: 0.0720 - mean_absolute_error: 0.0441
Epoch 6/50
868/868 [==============================] - 2s 2ms/step - loss: 0.4340 - root_mean_squared_error: 0.0680 - mean_absolute_error: 0.0398
Epoch 7/50
868/868 [==============================] - 2s 2ms/step - loss: 0.4277 - root_mean_squared_error: 0.0536 - mean_absolute

In [18]:
print(r2_score(y_test, y_pred1_2A))

0.9949939324659955


In [19]:
print(f" the predicted values for the mlp model is: {y_pred1_2A}")

 the predicted values for the mlp model is: [[0.83676994]
 [0.00941123]
 [0.96175057]
 [0.96668375]
 [0.9579214 ]
 [0.9517357 ]
 [0.06866314]
 [0.97992855]
 [0.32715115]
 [0.90594476]
 [0.98645943]
 [0.51029193]
 [0.14088237]
 [0.11617707]
 [0.09134915]
 [0.5348726 ]
 [0.3963589 ]
 [0.19857106]
 [0.516536  ]
 [0.05628487]
 [0.61575913]
 [0.00449654]
 [0.06064405]
 [0.78858197]
 [0.03365729]
 [0.22461088]
 [0.08761397]
 [0.9908994 ]
 [0.01301818]
 [0.65169674]
 [0.2808165 ]
 [0.17624584]
 [0.9320387 ]
 [0.04936955]
 [0.98421234]
 [0.21482651]
 [0.10265019]
 [0.17790048]
 [0.12977862]
 [0.3352997 ]
 [0.2538849 ]
 [0.18488662]
 [0.14185092]
 [0.97194767]]


In [31]:
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [32]:
import rpy2

In [33]:
%%R
install.package("float")


  could not find function "install.package"




Error in install.package("float") : 
  could not find function "install.package"


RInterpreterError: ignored

In [ ]:
actual_result= robjects.FloatVector({0.78256331, 0.01013892, 0.93909211, 0.93171893, 0.97838549, 0.9377304, 0.05729306, 0.96406324, 0.30674558, 0.88423513, 0.98838939, 0.52518533, 0.15899424, 0.10595266, 0.08948024, 0.49998389, 0.36971785, 0.22523765, 0.604794, 0.04048216, 0.55971815, 0.00409338, 0.06031447, 0.832428, 0.02997866, 0.26498554, 0.07152355, 0.9884246, 0.0168044, 0.64804015, 0.2965188, 0.14913114, 0.9497351, 0.05940566, 0.98500799, 0.22879337,  0.09548048, 0.19285071, 0.14934963, 0.33025048, 0.22191112, 0.15581709,  0.14532238, 0.98067324})


In [ ]:
d = {'xgboost': robjects.IntVector(), 'rbf': robjects.IntVector(), 'mlp':robjects.IntVector()}
dataf = robject.DataFrame(d)